<a href="https://colab.research.google.com/github/kashishthakur26/Data-Science/blob/main/SBERT_NLP_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentence Embedding and Similairty

---
Pretrained Models

https://www.sbert.net/docs/pretrained_models.html


In [7]:
%pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=66339416b6fbb3ce6c83982febd6e5e1bd31cc4101f0776747a01a9cfd84c42f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [8]:
from sentence_transformers import SentenceTransformer , util

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
sentences = ['the cat sites outside' ,'the new movie is awesome','the new movie is really great', 'the dog bark on strangers']


In [11]:
embeddings = model.encode(sentences =sentences , convert_to_tensor =True)


In [12]:
for sent , embed in zip(sentences ,embeddings):
  print('Sentence:' , sent)
  print('len(Embeddings)' , len(embed))

Sentence: the cat sites outside
len(Embeddings) 384
Sentence: the new movie is awesome
len(Embeddings) 384
Sentence: the new movie is really great
len(Embeddings) 384
Sentence: the dog bark on strangers
len(Embeddings) 384


In [13]:
cosine_scores = util.cos_sim(embeddings , embeddings)


In [14]:
cosine_scores

tensor([[1.0000, 0.0193, 0.0044, 0.2035],
        [0.0193, 1.0000, 0.9074, 0.1454],
        [0.0044, 0.9074, 1.0000, 0.1662],
        [0.2035, 0.1454, 0.1662, 1.0000]])

In [15]:
sentences

['the cat sites outside',
 'the new movie is awesome',
 'the new movie is really great',
 'the dog bark on strangers']

In [16]:
paraphrases = util.paraphrase_mining(model , sentences)

In [19]:
for sim in paraphrases[0:10]:
  score , i  , j = sim
  print(sentences[i] ,"<>" , sentences[j] ,"-->",score)

the new movie is awesome <> the new movie is really great --> 0.9074223041534424
the cat sites outside <> the dog bark on strangers --> 0.2035394161939621
the new movie is really great <> the dog bark on strangers --> 0.16619452834129333
the new movie is awesome <> the dog bark on strangers --> 0.14539891481399536
the cat sites outside <> the new movie is awesome --> 0.019304918125271797
the cat sites outside <> the new movie is really great --> 0.00441988930106163


## Semantic Search

In [20]:
import torch

In [21]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [22]:
import requests

In [24]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/master/text-dataset-for-machine-learning/sbert-corpus.txt')
corpus = response.text.split('\r\n')

In [25]:
corpus

['A man is eating food.',
 'A man is eating a piece of bread.',
 'A man is eating pasta.',
 'The girl is carrying a baby.',
 'The baby is carried by the woman',
 'A man is riding a horse.',
 'A man is riding a white horse on an enclosed ground.',
 'A monkey is playing drums.',
 'Someone in a gorilla costume is playing a set of drums.',
 'A cheetah is running behind its prey.',
 'A cheetah chases prey on across a field.']

In [26]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/master/text-dataset-for-machine-learning/sbert-queries.txt')
queries = response.text.split('\r\n')
queries

['A man is eating pasta.',
 'Someone in a gorilla costume is playing a set of drums.',
 'A cheetah chases prey on across a field.']

In [27]:
corpus_embeddings = model.encode(corpus , convert_to_tensor=True)
queries_embeddings = model.encode(queries , convert_to_tensor=True)

In [28]:
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)
queries_embeddings = util.normalize_embeddings(queries_embeddings)

In [29]:
len(corpus_embeddings[0])

384

In [30]:
hits = util.semantic_search(queries_embeddings , corpus_embeddings ,score_function = util.dot_score , top_k =3)


In [31]:
hits

[[{'corpus_id': 2, 'score': 0.9999999403953552},
  {'corpus_id': 0, 'score': 0.8384666442871094},
  {'corpus_id': 1, 'score': 0.7468275427818298}],
 [{'corpus_id': 8, 'score': 1.0},
  {'corpus_id': 7, 'score': 0.7612735033035278},
  {'corpus_id': 3, 'score': 0.3815288543701172}],
 [{'corpus_id': 10, 'score': 1.0},
  {'corpus_id': 9, 'score': 0.8703994750976562},
  {'corpus_id': 6, 'score': 0.37411707639694214}]]

In [32]:
for query, hit in zip(queries, hits):
  for q_hit in hit:
    id = q_hit['corpus_id']
    score = q_hit['score']

    print(query, "<>", corpus[id], "-->", score)

  print()

A man is eating pasta. <> A man is eating pasta. --> 0.9999999403953552
A man is eating pasta. <> A man is eating food. --> 0.8384666442871094
A man is eating pasta. <> A man is eating a piece of bread. --> 0.7468275427818298

Someone in a gorilla costume is playing a set of drums. <> Someone in a gorilla costume is playing a set of drums. --> 1.0
Someone in a gorilla costume is playing a set of drums. <> A monkey is playing drums. --> 0.7612735033035278
Someone in a gorilla costume is playing a set of drums. <> The girl is carrying a baby. --> 0.3815288543701172

A cheetah chases prey on across a field. <> A cheetah chases prey on across a field. --> 1.0
A cheetah chases prey on across a field. <> A cheetah is running behind its prey. --> 0.8703994750976562
A cheetah chases prey on across a field. <> A man is riding a white horse on an enclosed ground. --> 0.37411707639694214



## Clustering
Sentence-Transformers can be used in different ways to perform clustering of small or large set of sentences.

### K-Mean Clustering

In [34]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

In [35]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [36]:
import requests

In [37]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/master/text-dataset-for-machine-learning/sbert-corpus.txt')
corpus = response.text.split('\r\n')

In [38]:
len(corpus), print(corpus)

['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.', 'The girl is carrying a baby.', 'The baby is carried by the woman', 'A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.', 'A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']


(11, None)

In [39]:
corpus_embeddings = model.encode(corpus)

In [40]:
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [41]:
cluster_assignment

array([0, 0, 0, 2, 2, 3, 3, 4, 4, 1, 1], dtype=int32)

In [42]:
clustered_sentences = [[] for i in range(num_clusters)]
clustered_sentences

[[], [], [], [], []]

In [43]:
for sentence_id, cluster_id in enumerate(cluster_assignment):
  clustered_sentences[cluster_id].append(corpus[sentence_id])

In [44]:
for i, cluster in enumerate(clustered_sentences):
  print("Cluster ", i+1)
  print(cluster)
  print()

Cluster  1
['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.']

Cluster  2
['A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']

Cluster  3
['The girl is carrying a baby.', 'The baby is carried by the woman']

Cluster  4
['A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.']

Cluster  5
['A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.']



### agglomerative Clustering

It is a type of hierarchical clustering.The hierarchical clustering analysis is a method of cluster analysis that seeks to build a hierarchy of clusters i.e tree-type structure based on the hierarchy.

In [46]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [47]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [48]:
print(corpus)

['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.', 'The girl is carrying a baby.', 'The baby is carried by the woman', 'A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.', 'A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']


In [49]:
corpus_embeddings = model.encode(corpus)

In [50]:
corpus_embeddings = corpus_embeddings/np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

In [51]:
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [52]:
cluster_assignment

array([0, 0, 0, 4, 4, 1, 1, 2, 2, 3, 3])

In [53]:
num_clusters = len(np.unique(cluster_assignment))
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
  clustered_sentences[cluster_id].append(corpus[sentence_id])


for i, cluster in enumerate(clustered_sentences):
  print("Cluster ", i+1)
  print(cluster)
  print()


Cluster  1
['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.']

Cluster  2
['A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.']

Cluster  3
['A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.']

Cluster  4
['A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']

Cluster  5
['The girl is carrying a baby.', 'The baby is carried by the woman']



### Fast Clustering
Algomerative Clustering for larger datasets is quite slow , so it is only applicable for maybe a few thousand sentences.
Fast Clustering can fine-tune 50k sentences withing few seconds. You can use threshold to define clustering criteria.

In [54]:
from sentence_transformers import SentenceTransformer, util

import pandas as pd
import time

In [55]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [62]:
df = pd.read_csv("http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv", sep='\t')
df.shape

HTTPError: ignored